In [1]:
!pip install qiskit

In [2]:
from qiskit import QuantumCircuit
import qiskit.quantum_info as qi
import numpy as np

# Revers(ibl)e Engineering [0/2]
## Présentation de l'énoncé
### Enoncé
"My game is a lot about footwork. If I move well - I play well." - Joueur de tennis peu connu

Au tennis, des déplacements optimaux font gagner des matchs - même chose ici, mais avec des circuits logiques. Recevez un circuit logique réversible effectuant une opération sur 3 bits. Renvoyez un circuit optimal équivalent. Plusieurs solutions sont possibles.


Précisions :
Les seules portes logiques réversibles utilisées seront :
La porte NOT
La porte CNOT (controlled NOT)
La porte TOFFOLI
La convention de notation sera de noter les bits de contrôle en premier.
Deux circuits seront dit équivalents s'ils effectuent la même opération.
Un circuit sera dit optimal s'il n'existe aucun circuit équivalent comportant un nombre de portes logiques strictement plus petit.
Si la connexion se coupe sans message, c'est que vous avez timeout ! Pas plus de 60 secondes par circuit !

### Exemple de cicuit
Un exemple de circuit à optimiser est ```{"gates": [["NOT", [0]], ["CNOT", [2, 0]], ["CNOT", [1, 2]], ["NOT", [2]], ["CNOT", [2, 0]], ["TOFFOLI", [2, 1, 0]], ["NOT", [2]], ["NOT", [0]]], "bits": 3}```

### Hypothèses complémentaire
J'ai constaté que le serveur envoie uniquement des portes à 3 qubits. Dans la suite, j'ai hardcodé cette valeur.

## Introduction à Qiskit

Dans la suite, je vais utiliser Qiskit pour manipuler les portes logiques et les circuits donc je vous propose une rapide introduction à l'outil dans cette section.

On considère le circut suivant : ```[["NOT", [0]], ["CNOT", [2, 0]], ["CNOT", [1, 2]], ["NOT", [2]], ["CNOT", [2, 0]], ["TOFFOLI", [2, 1, 0]], ["NOT", [2]], ["NOT", [0]]]```. On peut le représenter dans Qiskit avec :

In [3]:
qc = QuantumCircuit(3) # Création d'un circuit à 3 Qubits
qc.x(0)         # NOT(0)
qc.cx(2,0)      # CNOT(2,0)
qc.cx(1,2)      # CNOT(1,2)
qc.x(2)         # NOT(2)
qc.cx(2,0)      # NOT(2,0)
qc.ccx(2,1,0)   # TOFFOLI(2,1,0)
qc.x(2)         # NOT(2)
qc.x(0)         # NOT(0)

qc.draw()       # Visualiser le circuit

┌───┐┌───┐          ┌───┐┌───┐┌───┐
q_0: ┤ X ├┤ X ├──────────┤ X ├┤ X ├┤ X ├
     └───┘└─┬─┘          └─┬─┘└─┬─┘└───┘
q_1: ───────┼────■─────────┼────■───────
            │  ┌─┴─┐┌───┐  │    │  ┌───┐
q_2: ───────■──┤ X ├┤ X ├──■────■──┤ X ├
               └───┘└───┘          └───┘

On peut également visualiser la matrice associée au circuit

In [4]:
qc_matrix = qi.Operator(qc).data
print(qc_matrix)

[[0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


Les coefficients de la matrice étant rééls, pour plus de lisibilité, on pourra utiliser la fonction suivante



In [5]:
def cmplx_to_real_array(arr):
  """
  Parce que c'est plus facile à observer
  """
  res = np.zeros(arr.shape)
  for i in range(0,res.shape[0]):
    for j in range(0,res.shape[1]):
      if arr[i][j].imag != 0:
        raise Exception("Some coeff is complex")
      res[i][j] = arr[i][j].real
  return res

print(cmplx_to_real_array(qc_matrix))

[[0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]]


L'intérêt d'utiliser Qiskit est la capacité de facilement manipuler les circuits et les portes sous forme matricielle avec [qiskit.quantum_info.Operator](https://docs.quantum.ibm.com/api/qiskit/qiskit.quantum_info.Operator)

In [6]:
def gen_circuit_from_ctf(inp) -> QuantumCircuit:
  """
  Fonction permettant d'automatiser la génération d'un circuit à partir d'un input de la forme :
  # INPUT = [["NOT", [0]], ["CNOT", [2, 0]], ["CNOT", [1, 2]], ["NOT", [2]], ["CNOT", [2, 0]], ["TOFFOLI", [2, 1, 0]], ["NOT", [2]], ["NOT", [0]]]
  """
  qc = QuantumCircuit(3)
  for gate in inp:
    if gate[0] == "NOT":
      qc.x(gate[1][0])
    elif gate[0] == "CNOT":
      qc.cx(gate[1][0], gate[1][1])
    elif gate[0] == "TOFFOLI":
      qc.ccx(gate[1][0],gate[1][1],gate[1][2])
    else:
      assert False
  return qc

 ## Les portes ```NOT```, ```CNOT``` et ```TOFFOLI```
 Le sujet nous indique que les portes ```NOT```, ```CNOT``` et ```TOFFOLI``` sont réversibles.

 On va observer à l'aide de Qiskit que ces portes sont représentées par des [matrices de permutation](https://fr.wikipedia.org/wiki/Matrice_de_permutation).

 Mais mieux encore, on va également remarquer que la matrice associée à chacune des portes a pour inverse elle-même. Ce qui s'intuite assez facilement : NOT(NOT(x)) == X...



In [7]:
# Génération des matrices des différentes portes NOT possibles
# NOT(0)
NOT0_circ = QuantumCircuit(3)
NOT0_circ.x(0)
# NOT(1)
NOT1_circ = QuantumCircuit(3)
NOT1_circ.x(1)
# NOT(2)
NOT2_circ = QuantumCircuit(3)
NOT2_circ.x(2)

NOT0_M = qi.Operator(NOT0_circ)
NOT1_M = qi.Operator(NOT1_circ)
NOT2_M = qi.Operator(NOT2_circ)

assert ((NOT0_M.dot(NOT0_M).data)==np.identity(8)).all()

In [8]:
# Génération des matrices des différentes portes CNOT possibles
# CNOT(0, 1)
CNOT01_circ = QuantumCircuit(3)
CNOT01_circ.cx(0, 1)
# CNOT(0, 2)
CNOT02_circ = QuantumCircuit(3)
CNOT02_circ.cx(0, 2)
# CNOT(1, 0)
CNOT10_circ = QuantumCircuit(3)
CNOT10_circ.cx(1, 0)
# CNOT(1, 2)
CNOT12_circ = QuantumCircuit(3)
CNOT12_circ.cx(1, 2)
# CNOT(2, 0)
CNOT20_circ = QuantumCircuit(3)
CNOT20_circ.cx(2, 0)
# CNOT(2, 1)
CNOT21_circ = QuantumCircuit(3)
CNOT21_circ.cx(2, 1)

CNOT01_M = qi.Operator(CNOT01_circ)
CNOT02_M = qi.Operator(CNOT02_circ)
CNOT10_M = qi.Operator(CNOT10_circ)
CNOT12_M = qi.Operator(CNOT12_circ)
CNOT20_M = qi.Operator(CNOT20_circ)
CNOT21_M = qi.Operator(CNOT21_circ)

assert ((CNOT01_M.dot(CNOT01_M).data)==np.identity(8)).all()

In [9]:
# Génération des matrices des différentes portes TOFFOLI possibles
# TOFFOLI(0, 1, 2) == TOFFOLI(1, 0, 2)
TOFF2_circ = QuantumCircuit(3)
TOFF2_circ.ccx(0, 1, 2)
# TOFFOLI(1, 2, 0) == TOFFOLI(2, 1, 0)
TOFF0_circ = QuantumCircuit(3)
TOFF0_circ.ccx(1, 2, 0)
# TOFFOLI(0, 2, 1) == TOFFOLI(2, 0, 1)
TOFF1_circ = QuantumCircuit(3)
TOFF1_circ.ccx(0, 2, 1)

TOFF0_M = qi.Operator(TOFF0_circ)
TOFF1_M = qi.Operator(TOFF1_circ)
TOFF2_M = qi.Operator(TOFF2_circ)

assert ((TOFF2_M.dot(TOFF2_M).data)==np.identity(8)).all()

### Stratégie et implémentation
A partir du circuit reçu, on récupère la matrice de permutation associée.
Puis on va essayer de factoriser cette matrice avec pour facteur les matrices des portes ```NOT```, ```CNOT``` et ```TOFFOLI```.

In [10]:
# Dictionnaire contenant toutes les portes pouvant être utilisées lors de la factorisation
BASIC_MX_DICT = {
    "NOT0_M": [NOT0_M, ["NOT", [0]]],
    "NOT1_M": [NOT1_M, ["NOT", [1]]],
    "NOT2_M": [NOT2_M, ["NOT", [2]]],

    "CNOT01_M": [CNOT01_M, ["CNOT", [0, 1]]],
    "CNOT02_M": [CNOT02_M, ["CNOT", [0, 2]]],
    "CNOT10_M": [CNOT10_M, ["CNOT", [1, 0]]],
    "CNOT12_M": [CNOT12_M, ["CNOT", [1, 2]]],
    "CNOT20_M": [CNOT20_M, ["CNOT", [2, 0]]],
    "CNOT21_M": [CNOT21_M, ["CNOT", [2, 1]]],

    "TOFF0_M": [TOFF0_M, ["TOFFOLI", [1, 2, 0]]],
    "TOFF1_M": [TOFF1_M, ["TOFFOLI", [0, 2, 1]]],
    "TOFF2_M": [TOFF2_M, ["TOFFOLI", [1, 0, 2]]]
    }

Notons $M_{C}$ la matrice du circuit à optimiser. Je cherche une plus courte suite de portes $P_i \in \text{BASIC_MX_DICT}$ tel que $M_{C} = \prod_{i}P_i$. Pour reconstruire le circuit, il suffira de concaténer les portes dans l'ordre inverse : $P_kP_{k-1}...P_{0}$ car multiplier à droite revient à ajouter un élément en début de circuit.

En pratique, dans la suite, j'ai plutôt construit la suite des $P_i$ avec un algorithme de parcours en largeur avec pour condition d'arrêt $M_{C}\prod_{i}P_i = Id$. Ce qui revient au même mais pour reconstruire le circuit, il suffira de concaténer les portes dans l'ordre : $P_{0}...P_{k-1}P_k$.

In [11]:
def optimize_bfs(ctfMatrix, maxDepth):
  """
  Algorithme de parcours en largeur cherchant une factorisation optimale de ctfMatrix avec au plus "maxDepth" facteur.
  """
  # Initialisation avec l'hypothèse que le circuit optimisé comportera au moins 2 portes.
  initialState = [([k], ctfMatrix.dot(BASIC_MX_DICT[k][0])) for k in BASIC_MX_DICT]
  for i in range(1, maxDepth):
    currentState = []
    for s in initialState:
      for gate in BASIC_MX_DICT:
        curM = s[1].dot(BASIC_MX_DICT[gate][0])
        if (curM==np.identity(8)).all():
          return s[0] + [gate]
        currentState.append((s[0] + [gate], curM))
    initialState = currentState
  raise Exception("NOT FOUND: MaxDepth reached")

In [12]:
def bfs_opt_to_ctf(optiStates):
  """
  Fonction permettant de transformer le résultat retourné par optimize_bfs sous la forme d'une liste compatible avec la représentation du circuit utilisée par le 404CTF dans le champ "gates".
  """
  res = []
  for elt in optiStates:
    res.append(BASIC_MX_DICT[elt][1])
  return res

def bfs_opt_to_ctf_str(optiStates):
  """
  Fonction permettant de transformer le résultat retourné par optimize_bfs sous la forme d'une liste compatible avec la représentation utilisée par le 404CTF.
  """
  res = bfs_opt_to_ctf(optiStates)
  resStr = '{"gates": ' +str(res).replace("'",'"') + ', "bits": 3}'
  return resStr

## Tests

In [13]:
INPUT = [["CNOT", [2, 0]], ["CNOT", [2, 1]], ["NOT", [2]], ["CNOT", [2, 1]], ["TOFFOLI", [0, 2, 1]], ["CNOT", [1, 2]], ["NOT", [0]], ["CNOT", [0, 2]]]
qc404 = gen_circuit_from_ctf(INPUT)
qc404.draw() # Circuit non optimisé

┌───┐                    ┌───┐     
q_0: ┤ X ├─────────────────■──┤ X ├──■──
     └─┬─┘┌───┐     ┌───┐┌─┴─┐└───┘  │  
q_1: ──┼──┤ X ├─────┤ X ├┤ X ├──■────┼──
       │  └─┬─┘┌───┐└─┬─┘└─┬─┘┌─┴─┐┌─┴─┐
q_2: ──■────■──┤ X ├──■────■──┤ X ├┤ X ├
               └───┘          └───┘└───┘

In [14]:
# Optimisation
optimal = optimize_bfs(qi.Operator(qc404).data, len(INPUT))
print(bfs_opt_to_ctf_str(optimal)) # Résultat à renvoyer au serveur

{"gates": [["NOT", [0]], ["TOFFOLI", [0, 2, 1]], ["CNOT", [2, 0]], ["NOT", [2]], ["CNOT", [1, 2]], ["CNOT", [0, 1]]], "bits": 3}


On peut même visualiser le circuit optimisé

In [15]:
# Circuit optimisé
optimalCirc = gen_circuit_from_ctf(bfs_opt_to_ctf(optimal))
optimalCirc.draw()

┌───┐     ┌───┐               
q_0: ┤ X ├──■──┤ X ├────────────■──
     └───┘┌─┴─┐└─┬─┘          ┌─┴─┐
q_1: ─────┤ X ├──┼─────────■──┤ X ├
          └─┬─┘  │  ┌───┐┌─┴─┐└───┘
q_2: ───────■────■──┤ X ├┤ X ├─────
                    └───┘└───┘